# Домашнее задание 3. TensorFlow

In [510]:
#загрузка необходимых библиотек и модулей
from sklearn.datasets import load_boston

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import keras
import tensorflow as tf

from sklearn.model_selection import train_test_split

from itertools import product

from keras import backend as K
from keras.layers import Dense
from keras.models import Sequential
from tensorflow.keras.layers import Dense

In [511]:
def r_2_score(y_true, y_pred):
    """Вычисляет коэффициент детерминации r2. 
       Предназначена для использования в качестве метрики нейронной сети.
    """
    SS_res =  K.sum(K.square( y_true-y_pred )) 
    SS_tot = K.sum(K.square( y_true - K.mean(y_true) ) ) 
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

## Задание
  1. Постройте нейронную сеть (берем несложную полносвязную сеть, меняем число слоев, число нейронов, типы активации, тип оптимизатора) на датасете from sklearn.datasets import load_boston. 
  
  2. Измените функцию потерь и метрику для этой задачи. Постройте 10-15 вариантов и сведите результаты их работы в таблицу.  Опишите, какого результата вы добились от нейросети? Что помогло вам улучшить ее точность?

## План работы

[0. Загрузка данных](#section_0)

[1. Подготовка данных](#section_1)

[2. Двухслойные нейронные сети](#section_2)

[3. Трехслойные нейронные сети](#section_3)

[4. Сводная таблица метрик моделей и выводы](#section_4)

## 0. Загрузка данных<a id='section_0'></a>

In [512]:
#загрузка датасета
boston_dataset = load_boston()
print(boston_dataset.keys())

dict_keys(['data', 'target', 'feature_names', 'DESCR', 'filename'])


In [513]:
#описание датасета
print(boston_dataset.DESCR)

.. _boston_dataset:

Boston house prices dataset
---------------------------

**Data Set Characteristics:**  

    :Number of Instances: 506 

    :Number of Attributes: 13 numeric/categorical predictive. Median Value (attribute 14) is usually the target.

    :Attribute Information (in order):
        - CRIM     per capita crime rate by town
        - ZN       proportion of residential land zoned for lots over 25,000 sq.ft.
        - INDUS    proportion of non-retail business acres per town
        - CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
        - NOX      nitric oxides concentration (parts per 10 million)
        - RM       average number of rooms per dwelling
        - AGE      proportion of owner-occupied units built prior to 1940
        - DIS      weighted distances to five Boston employment centres
        - RAD      index of accessibility to radial highways
        - TAX      full-value property-tax rate per $10,000
        - PTRATIO  pu

In [514]:
#формирование датасета с признаками и целевой переменной
df = pd.DataFrame(boston_dataset.data, columns=boston_dataset.feature_names)
df['target'] = pd.Series(boston_dataset.target)

print(df.shape)
df.head()

(506, 14)


,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,target
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


## 1. Подготовка данных<a id='section_1'></a>

In [515]:
#Разведение признаков и целевой переменной по разным датасетам
X = df.drop(['target'], axis=1)
y = df['target']

In [516]:
#разделение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=1)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(354, 13)
(152, 13)
(354,)
(152,)


In [517]:
#нормализация данных (z-масштабирование)
mean = X_train.mean(axis=0)
std = X_train.std(axis=0)

X_train = (X_train - mean) / std
X_test = (X_test - mean) / std

## 2. Двухслойные нейронные сети<a id='section_2'></a>

In [518]:
#датасет для сбора результатов исследования двухслойных нейронных сетей
metric_nn_2 = pd.DataFrame(columns=['Layers', 
                                  'Layer_1',
                                  'Layer_2',
                                  'Act_func',
                                  'Loss',
                                  'Opt',
                                  'MSE_train',
                                  'MSE_test',
                                  'RMSE_train',
                                  'RMSE_test',
                                  'MAE_train',
                                  'MAE_test',
                                  'R2_score_train',
                                  'R2_score_test']) 

In [519]:
%%time

#списки значений переменных
dense_1_neurons_list = [64,128]
activation_function_list = ['relu', 'linear']
loss_list = ['mse', 'mae']
optimizer_list = ['adam', 'sgd', 'rmsprop']

# словарь для хранения моделей
models_2 = {}


for (dense_1_num_neurons, act_func, loss, opt) in product(dense_1_neurons_list,activation_function_list,loss_list,optimizer_list):
  
    #инициализация модели
    model_2 = Sequential([
    Dense(dense_1_num_neurons,input_shape=(X_train.shape[1],), activation='relu', name='layer_1'),
    Dense(1, activation=act_func, name='layer_2')
                       ])

    #компиляция модели
    model_2.compile(loss=loss, 
               optimizer=opt, 
               metrics=[tf.keras.metrics.MeanSquaredError(),tf.keras.metrics.RootMeanSquaredError(),tf.keras.metrics.MeanAbsoluteError(), r_2_score])
              
    #обучение модели
    hist_2 = model_2.fit(X_train, y_train,
                   epochs=100,
                   batch_size=10,
                   verbose=1,
                   validation_data=(X_test, y_test))
    
    #оценка качества модели
    train_mse = hist_2.history['mean_squared_error'][-1]
    test_mse = hist_2.history['val_mean_squared_error'][-1]
    
    train_rmse = hist_2.history['root_mean_squared_error'][-1]
    test_rmse = hist_2.history['val_root_mean_squared_error'][-1]
    
    train_mae = hist_2.history['mean_absolute_error'][-1]
    test_mae = hist_2.history['val_mean_absolute_error'][-1]
    
    train_r2 = hist_2.history['r_2_score'][-1]
    test_r2 = hist_2.history['val_r_2_score'][-1]
    
    #сохранение обученной модели
    models_2[f'{dense_1_num_neurons}-{act_func}-{opt}'] = model_2
    
    #сохранение параметров и метрик модели в датафрейм
    metric_nn_2 = metric_nn_2.append({'Layers': 2, 
                                  'Layer_1': dense_1_num_neurons,
                                  'Layer_2': 'No',
                                  'Act_func': act_func,
                                  'Loss': loss,
                                  'Opt': opt,
                                  'MSE_train': train_mse,
                                  'MSE_test': test_mse,
                                  'RMSE_train':train_rmse,
                                  'RMSE_test': test_rmse,
                                  'MAE_train': train_mae,
                                  'MAE_test': test_mae,
                                  'R2_score_train': train_r2,
                                  'R2_score_test': test_r2},
                                  ignore_index=True)

Epoch 1/100
36/36 [==============================] - 0s 5ms/step - loss: 544.6575 - mean_squared_error: 544.6575 - root_mean_squared_error: 23.3379 - mean_absolute_error: 21.6395 - r_2_score: -7.7836 - val_loss: 555.5297 - val_mean_squared_error: 555.5297 - val_root_mean_squared_error: 23.5697 - val_mean_absolute_error: 21.6990 - val_r_2_score: -9.7961
Epoch 2/100
36/36 [==============================] - 0s 2ms/step - loss: 490.4182 - mean_squared_error: 490.4182 - root_mean_squared_error: 22.1454 - mean_absolute_error: 20.5069 - r_2_score: -16.6504 - val_loss: 493.7265 - val_mean_squared_error: 493.7265 - val_root_mean_squared_error: 22.2200 - val_mean_absolute_error: 20.3697 - val_r_2_score: -8.4228
Epoch 3/100
36/36 [==============================] - 0s 2ms/step - loss: 424.3178 - mean_squared_error: 424.3178 - root_mean_squared_error: 20.5990 - mean_absolute_error: 18.9951 - r_2_score: -6.2664 - val_loss: 419.2206 - val_mean_squared_error: 419.2206 - val_root_mean_squared_error: 20

36/36 [==============================] - 0s 2ms/step - loss: 24.0908 - mean_squared_error: 24.0908 - root_mean_squared_error: 4.9082 - mean_absolute_error: 3.5021 - r_2_score: 0.6369 - val_loss: 22.8299 - val_mean_squared_error: 22.8299 - val_root_mean_squared_error: 4.7781 - val_mean_absolute_error: 3.6055 - val_r_2_score: 0.7000
Epoch 25/100
36/36 [==============================] - 0s 2ms/step - loss: 23.6837 - mean_squared_error: 23.6837 - root_mean_squared_error: 4.8666 - mean_absolute_error: 3.4716 - r_2_score: 0.5288 - val_loss: 22.2265 - val_mean_squared_error: 22.2265 - val_root_mean_squared_error: 4.7145 - val_mean_absolute_error: 3.5515 - val_r_2_score: 0.7049
Epoch 26/100
36/36 [==============================] - 0s 2ms/step - loss: 23.2441 - mean_squared_error: 23.2441 - root_mean_squared_error: 4.8212 - mean_absolute_error: 3.4497 - r_2_score: 0.6527 - val_loss: 22.0035 - val_mean_squared_error: 22.0035 - val_root_mean_squared_error: 4.6908 - val_mean_absolute_error: 3.5594

Epoch 48/100
36/36 [==============================] - 0s 2ms/step - loss: 15.7324 - mean_squared_error: 15.7324 - root_mean_squared_error: 3.9664 - mean_absolute_error: 2.8092 - r_2_score: 0.7096 - val_loss: 14.3493 - val_mean_squared_error: 14.3493 - val_root_mean_squared_error: 3.7881 - val_mean_absolute_error: 2.9380 - val_r_2_score: 0.7865
Epoch 49/100
36/36 [==============================] - 0s 2ms/step - loss: 15.5551 - mean_squared_error: 15.5551 - root_mean_squared_error: 3.9440 - mean_absolute_error: 2.7913 - r_2_score: 0.7585 - val_loss: 14.0941 - val_mean_squared_error: 14.0941 - val_root_mean_squared_error: 3.7542 - val_mean_absolute_error: 2.9112 - val_r_2_score: 0.7883
Epoch 50/100
36/36 [==============================] - 0s 2ms/step - loss: 15.2510 - mean_squared_error: 15.2510 - root_mean_squared_error: 3.9053 - mean_absolute_error: 2.7466 - r_2_score: 0.7528 - val_loss: 13.7389 - val_mean_squared_error: 13.7389 - val_root_mean_squared_error: 3.7066 - val_mean_absolute_

Epoch 72/100
36/36 [==============================] - 0s 2ms/step - loss: 11.7882 - mean_squared_error: 11.7882 - root_mean_squared_error: 3.4334 - mean_absolute_error: 2.3436 - r_2_score: 0.8028 - val_loss: 10.1168 - val_mean_squared_error: 10.1168 - val_root_mean_squared_error: 3.1807 - val_mean_absolute_error: 2.4871 - val_r_2_score: 0.8208
Epoch 73/100
36/36 [==============================] - 0s 2ms/step - loss: 11.6645 - mean_squared_error: 11.6645 - root_mean_squared_error: 3.4153 - mean_absolute_error: 2.3161 - r_2_score: 0.8102 - val_loss: 10.0897 - val_mean_squared_error: 10.0897 - val_root_mean_squared_error: 3.1764 - val_mean_absolute_error: 2.4909 - val_r_2_score: 0.8197
Epoch 74/100
36/36 [==============================] - 0s 2ms/step - loss: 11.5351 - mean_squared_error: 11.5351 - root_mean_squared_error: 3.3963 - mean_absolute_error: 2.3240 - r_2_score: 0.7765 - val_loss: 9.7959 - val_mean_squared_error: 9.7959 - val_root_mean_squared_error: 3.1298 - val_mean_absolute_er

Epoch 96/100
36/36 [==============================] - 0s 2ms/step - loss: 9.9396 - mean_squared_error: 9.9396 - root_mean_squared_error: 3.1527 - mean_absolute_error: 2.1217 - r_2_score: 0.8091 - val_loss: 8.6256 - val_mean_squared_error: 8.6256 - val_root_mean_squared_error: 2.9369 - val_mean_absolute_error: 2.3199 - val_r_2_score: 0.8267
Epoch 97/100
36/36 [==============================] - 0s 2ms/step - loss: 9.8743 - mean_squared_error: 9.8743 - root_mean_squared_error: 3.1423 - mean_absolute_error: 2.1163 - r_2_score: 0.8286 - val_loss: 8.6683 - val_mean_squared_error: 8.6683 - val_root_mean_squared_error: 2.9442 - val_mean_absolute_error: 2.3269 - val_r_2_score: 0.8294
Epoch 98/100
36/36 [==============================] - 0s 2ms/step - loss: 9.7998 - mean_squared_error: 9.7998 - root_mean_squared_error: 3.1305 - mean_absolute_error: 2.1083 - r_2_score: 0.8050 - val_loss: 8.5516 - val_mean_squared_error: 8.5516 - val_root_mean_squared_error: 2.9243 - val_mean_absolute_error: 2.314

...

Epoch 1/100
36/36 [==============================] - 0s 5ms/step - loss: 21.0662 - mean_squared_error: 521.2773 - root_mean_squared_error: 22.8315 - mean_absolute_error: 21.0662 - r_2_score: -9.7468 - val_loss: 20.7295 - val_mean_squared_error: 518.4148 - val_root_mean_squared_error: 22.7687 - val_mean_absolute_error: 20.7295 - val_r_2_score: -9.1180
Epoch 2/100
36/36 [==============================] - 0s 2ms/step - loss: 18.9525 - mean_squared_error: 435.9599 - root_mean_squared_error: 20.8797 - mean_absolute_error: 18.9525 - r_2_score: -7.7047 - val_loss: 18.4226 - val_mean_squared_error: 429.4299 - val_root_mean_squared_error: 20.7227 - val_mean_absolute_error: 18.4226 - val_r_2_score: -7.1686
Epoch 3/100
36/36 [==============================] - 0s 2ms/step - loss: 16.4218 - mean_squared_error: 342.8994 - root_mean_squared_error: 18.5175 - mean_absolute_error: 16.4218 - r_2_score: -5.7894 - val_loss: 15.6985 - val_mean_squared_error: 333.6409 - val_root_mean_squared_error: 18.2658 -

Epoch 25/100
36/36 [==============================] - 0s 1ms/step - loss: 2.7271 - mean_squared_error: 18.7962 - root_mean_squared_error: 4.3355 - mean_absolute_error: 2.7271 - r_2_score: 0.7466 - val_loss: 2.8186 - val_mean_squared_error: 15.0493 - val_root_mean_squared_error: 3.8793 - val_mean_absolute_error: 2.8186 - val_r_2_score: 0.7888
Epoch 26/100
36/36 [==============================] - 0s 2ms/step - loss: 2.6967 - mean_squared_error: 18.5546 - root_mean_squared_error: 4.3075 - mean_absolute_error: 2.6967 - r_2_score: 0.7187 - val_loss: 2.7631 - val_mean_squared_error: 14.8934 - val_root_mean_squared_error: 3.8592 - val_mean_absolute_error: 2.7631 - val_r_2_score: 0.7803
Epoch 27/100
36/36 [==============================] - 0s 1ms/step - loss: 2.6355 - mean_squared_error: 18.1297 - root_mean_squared_error: 4.2579 - mean_absolute_error: 2.6355 - r_2_score: 0.7394 - val_loss: 2.7916 - val_mean_squared_error: 14.4967 - val_root_mean_squared_error: 3.8074 - val_mean_absolute_error:

Epoch 49/100
36/36 [==============================] - 0s 2ms/step - loss: 2.1313 - mean_squared_error: 14.4653 - root_mean_squared_error: 3.8033 - mean_absolute_error: 2.1313 - r_2_score: 0.7763 - val_loss: 2.2616 - val_mean_squared_error: 10.0861 - val_root_mean_squared_error: 3.1759 - val_mean_absolute_error: 2.2616 - val_r_2_score: 0.8208
Epoch 50/100
36/36 [==============================] - 0s 2ms/step - loss: 2.1261 - mean_squared_error: 14.5202 - root_mean_squared_error: 3.8105 - mean_absolute_error: 2.1261 - r_2_score: 0.7889 - val_loss: 2.2569 - val_mean_squared_error: 9.9362 - val_root_mean_squared_error: 3.1522 - val_mean_absolute_error: 2.2569 - val_r_2_score: 0.8315
Epoch 51/100
36/36 [==============================] - 0s 2ms/step - loss: 2.1069 - mean_squared_error: 14.2519 - root_mean_squared_error: 3.7752 - mean_absolute_error: 2.1069 - r_2_score: 0.8186 - val_loss: 2.2849 - val_mean_squared_error: 10.0057 - val_root_mean_squared_error: 3.1632 - val_mean_absolute_error: 

Epoch 73/100
36/36 [==============================] - 0s 1ms/step - loss: 1.9798 - mean_squared_error: 13.0389 - root_mean_squared_error: 3.6109 - mean_absolute_error: 1.9798 - r_2_score: 0.8123 - val_loss: 2.1733 - val_mean_squared_error: 9.0588 - val_root_mean_squared_error: 3.0098 - val_mean_absolute_error: 2.1733 - val_r_2_score: 0.8318
Epoch 74/100
36/36 [==============================] - 0s 1ms/step - loss: 1.9829 - mean_squared_error: 12.9752 - root_mean_squared_error: 3.6021 - mean_absolute_error: 1.9829 - r_2_score: 0.7609 - val_loss: 2.1672 - val_mean_squared_error: 9.2186 - val_root_mean_squared_error: 3.0362 - val_mean_absolute_error: 2.1672 - val_r_2_score: 0.8327
Epoch 75/100
36/36 [==============================] - 0s 1ms/step - loss: 1.9513 - mean_squared_error: 13.0430 - root_mean_squared_error: 3.6115 - mean_absolute_error: 1.9513 - r_2_score: 0.7893 - val_loss: 2.1723 - val_mean_squared_error: 9.1074 - val_root_mean_squared_error: 3.0178 - val_mean_absolute_error: 2.

Epoch 97/100
36/36 [==============================] - 0s 1ms/step - loss: 1.8585 - mean_squared_error: 12.2728 - root_mean_squared_error: 3.5033 - mean_absolute_error: 1.8585 - r_2_score: 0.8465 - val_loss: 2.1462 - val_mean_squared_error: 8.8745 - val_root_mean_squared_error: 2.9790 - val_mean_absolute_error: 2.1462 - val_r_2_score: 0.8413
Epoch 98/100
36/36 [==============================] - 0s 1ms/step - loss: 1.8631 - mean_squared_error: 12.3265 - root_mean_squared_error: 3.5109 - mean_absolute_error: 1.8631 - r_2_score: 0.8355 - val_loss: 2.2010 - val_mean_squared_error: 9.0032 - val_root_mean_squared_error: 3.0005 - val_mean_absolute_error: 2.2010 - val_r_2_score: 0.8458
Epoch 99/100
36/36 [==============================] - 0s 1ms/step - loss: 1.8566 - mean_squared_error: 12.1075 - root_mean_squared_error: 3.4796 - mean_absolute_error: 1.8566 - r_2_score: 0.8342 - val_loss: 2.1511 - val_mean_squared_error: 8.7739 - val_root_mean_squared_error: 2.9621 - val_mean_absolute_error: 2.

In [521]:
#сводная таблица метрик двухуровневых моделей
metric_nn_2

,Layers,Layer_1,Layer_2,Act_func,Loss,Opt,MSE_train,MSE_test,RMSE_train,RMSE_test,MAE_train,MAE_test,R2_score_train,R2_score_test
0,2,64,No,relu,mse,adam,9.707432,8.475453,3.115675,2.911263,2.092662,2.303642,0.710858,0.827906
1,2,64,No,relu,mse,sgd,4.467323,14.176019,2.113604,3.765105,1.540355,3.011783,0.896765,0.712268
2,2,64,No,relu,mse,rmsprop,9.727237,8.184796,3.118852,2.860908,1.996526,2.221954,0.853504,0.831625
3,2,64,No,relu,mae,adam,12.881967,9.667577,3.589146,3.109273,1.927246,2.194268,0.829186,0.837214
4,2,64,No,relu,mae,sgd,12.286271,8.743618,3.505178,2.956961,1.837480,2.150621,0.808807,0.838279
5,2,64,No,relu,mae,rmsprop,13.050484,10.123955,3.612545,3.181816,1.998049,2.312062,0.817319,0.821419
6,2,64,No,linear,mse,adam,9.708630,8.518790,3.115867,2.918697,2.125846,2.309707,0.737283,0.825484
7,2,64,No,linear,mse,sgd,6.486161,8.509377,2.546794,2.917084,1.883709,2.165311,0.890590,0.811342
8,2,64,No,linear,mse,rmsprop,9.621620,7.839445,3.101874,2.799901,2.020330,2.184177,0.828677,0.830811
9,2,64,No,linear,mae,adam,12.059042,8.925554,3.472613,2.987567,1.855079,2.156638,0.806480,0.835851


In [522]:
#лучший показатель MSE на тесте
metric_nn_2.sort_values(by='MSE_test').head(1)

,Layers,Layer_1,Layer_2,Act_func,Loss,Opt,MSE_train,MSE_test,RMSE_train,RMSE_test,MAE_train,MAE_test,R2_score_train,R2_score_test
14,2,128,No,relu,mse,rmsprop,8.830073,7.485871,2.971544,2.736032,1.99523,2.140318,0.830675,0.834409


In [524]:
#лучший показатель RMSE на тесте
metric_nn_2.sort_values(by='RMSE_test').head(1)

,Layers,Layer_1,Layer_2,Act_func,Loss,Opt,MSE_train,MSE_test,RMSE_train,RMSE_test,MAE_train,MAE_test,R2_score_train,R2_score_test
14,2,128,No,relu,mse,rmsprop,8.830073,7.485871,2.971544,2.736032,1.99523,2.140318,0.830675,0.834409


In [525]:
#лучший показатель MAE на тесте
metric_nn_2.sort_values(by='MAE_test').head(1)

,Layers,Layer_1,Layer_2,Act_func,Loss,Opt,MSE_train,MSE_test,RMSE_train,RMSE_test,MAE_train,MAE_test,R2_score_train,R2_score_test
21,2,128,No,linear,mae,adam,10.758825,8.309279,3.280065,2.882582,1.733981,2.110387,0.83336,0.838112


In [526]:
#лучший показатель r2_score на тесте
metric_nn_2.sort_values(by='R2_score_test', ascending=False).head(1)

,Layers,Layer_1,Layer_2,Act_func,Loss,Opt,MSE_train,MSE_test,RMSE_train,RMSE_test,MAE_train,MAE_test,R2_score_train,R2_score_test
16,2,128,No,relu,mae,sgd,11.321071,8.871888,3.36468,2.978571,1.820883,2.225218,0.810803,0.851766


**Вывод:** Среди двухуровневых моделей:
1. Лучший показатель MSE имеет модель с 128 нейронами, функцией активации relu, функцией ошибок mse и оптимизатором rmsprop.
2. Лучший показатель RMSE имеет модель с 128 нейронами, функцией активации relu, функцией ошибок mse и оптимизатором rmsprop.
3. Лучший показатель MAE имеет модель с 128 нейронами, функцией активации linear, функцией ошибок mae и оптимизатором adam.
4. Лучшую метрику r2_score имеет модель с 128 нейронами, функцией активации relu, функцией ошибок mae и оптимизатором SGD.

## 3. Трехслойные нейронные сети<a id='section_3'></a>

In [532]:
#датасет для сбора результатов исследования трехслойных нейронных сетей
metric_nn_3 = pd.DataFrame(columns=['Layers', 
                                  'Layer_1',
                                  'Layer_2',
                                  'Act_func',
                                  'Loss',
                                  'Opt',
                                  'MSE_train',
                                  'MSE_test',
                                  'RMSE_train',
                                  'RMSE_test',
                                  'MAE_train',
                                  'MAE_test',
                                  'R2_score_train',
                                  'R2_score_test']) 

In [533]:
%%time

#список значений переменных
dense_1_neurons_list = [128,64]
dense_2_neurons_list = [64,32]
activation_function_list = ['relu', 'linear']
loss_list = ['mse', 'mae']
optimizer_list = ['adam', 'sgd', 'rmsprop']


#словарь для хранения моделей
models_3 = {}

for (dense_1_num_neurons, dense_2_num_neurons, act_func, loss, opt) in product(dense_1_neurons_list, dense_2_neurons_list, activation_function_list,loss_list,optimizer_list):
  
    #инициализация модели
    model_3 = Sequential([
    Dense(dense_1_num_neurons, input_shape=(X_train.shape[1],), activation='relu', name='layer_1'),
    Dense(dense_2_num_neurons, activation='relu', name='layer_2'),
    Dense(1, activation=act_func, name='layer_3')
                       ])

    #компиляция модели
    model_3.compile(loss=loss, 
               optimizer=opt, 
               metrics=[tf.keras.metrics.MeanSquaredError(),tf.keras.metrics.RootMeanSquaredError(),tf.keras.metrics.MeanAbsoluteError(), r_2_score])
              
    #обучение модели
    hist_3 = model_3.fit(X_train, y_train,
                   epochs=100,
                   batch_size=10,
                   verbose=1,
                   validation_data=(X_test, y_test))
    
    #оценка качества модели
    train_mse = hist_3.history['mean_squared_error'][-1]
    test_mse = hist_3.history['val_mean_squared_error'][-1]
    
    train_rmse = hist_3.history['root_mean_squared_error'][-1]
    test_rmse = hist_3.history['val_root_mean_squared_error'][-1]
    
    train_mae = hist_3.history['mean_absolute_error'][-1]
    test_mae = hist_3.history['val_mean_absolute_error'][-1]
    
    train_r2 = hist_3.history['r_2_score'][-1]
    test_r2 = hist_3.history['val_r_2_score'][-1]
    
    #сохранение обученной модели в словарь
    models_3[f'{dense_1_num_neurons}-{dense_2_num_neurons}-{act_func}-{loss}-{opt}'] = model_3
    
    #сохранение параметров и метрик модели в датафрейм
    metric_nn_3 = metric_nn_3.append({'Layers': 3, 
                                  'Layer_1': dense_1_num_neurons,
                                  'Layer_2': dense_2_num_neurons,
                                  'Act_func': act_func,
                                  'Loss' : loss,
                                  'Opt': opt,
                                  'MSE_train': train_mse,
                                  'MSE_test': test_mse,
                                  'RMSE_train':train_rmse,
                                  'RMSE_test': test_rmse,
                                  'MAE_train': train_mae,
                                  'MAE_test': test_mae,
                                  'R2_score_train': train_r2,
                                  'R2_score_test': test_r2},
                                  ignore_index=True)

Epoch 1/100
36/36 [==============================] - 0s 5ms/step - loss: 479.0813 - mean_squared_error: 479.0813 - root_mean_squared_error: 21.8879 - mean_absolute_error: 19.8902 - r_2_score: -7.6862 - val_loss: 378.7200 - val_mean_squared_error: 378.7200 - val_root_mean_squared_error: 19.4607 - val_mean_absolute_error: 16.9304 - val_r_2_score: -6.0737
Epoch 2/100
36/36 [==============================] - 0s 2ms/step - loss: 189.1257 - mean_squared_error: 189.1257 - root_mean_squared_error: 13.7523 - mean_absolute_error: 10.9819 - r_2_score: -1.7724 - val_loss: 83.2856 - val_mean_squared_error: 83.2856 - val_root_mean_squared_error: 9.1261 - val_mean_absolute_error: 7.1120 - val_r_2_score: -0.2143
Epoch 3/100
36/36 [==============================] - 0s 2ms/step - loss: 53.6307 - mean_squared_error: 53.6307 - root_mean_squared_error: 7.3233 - mean_absolute_error: 5.3893 - r_2_score: 0.0677 - val_loss: 36.7131 - val_mean_squared_error: 36.7131 - val_root_mean_squared_error: 6.0591 - val_m

Epoch 25/100
36/36 [==============================] - 0s 2ms/step - loss: 10.9917 - mean_squared_error: 10.9917 - root_mean_squared_error: 3.3154 - mean_absolute_error: 2.2946 - r_2_score: 0.8383 - val_loss: 9.1969 - val_mean_squared_error: 9.1969 - val_root_mean_squared_error: 3.0326 - val_mean_absolute_error: 2.4179 - val_r_2_score: 0.8078
Epoch 26/100
36/36 [==============================] - 0s 2ms/step - loss: 10.7543 - mean_squared_error: 10.7543 - root_mean_squared_error: 3.2794 - mean_absolute_error: 2.2304 - r_2_score: 0.7857 - val_loss: 10.0367 - val_mean_squared_error: 10.0367 - val_root_mean_squared_error: 3.1681 - val_mean_absolute_error: 2.5392 - val_r_2_score: 0.7948
Epoch 27/100
36/36 [==============================] - 0s 2ms/step - loss: 10.3766 - mean_squared_error: 10.3766 - root_mean_squared_error: 3.2213 - mean_absolute_error: 2.2548 - r_2_score: 0.8400 - val_loss: 9.1874 - val_mean_squared_error: 9.1874 - val_root_mean_squared_error: 3.0311 - val_mean_absolute_erro

Epoch 49/100
36/36 [==============================] - 0s 2ms/step - loss: 7.4604 - mean_squared_error: 7.4604 - root_mean_squared_error: 2.7314 - mean_absolute_error: 1.9285 - r_2_score: 0.8425 - val_loss: 8.5885 - val_mean_squared_error: 8.5885 - val_root_mean_squared_error: 2.9306 - val_mean_absolute_error: 2.2788 - val_r_2_score: 0.7957
Epoch 50/100
36/36 [==============================] - 0s 2ms/step - loss: 7.6483 - mean_squared_error: 7.6483 - root_mean_squared_error: 2.7656 - mean_absolute_error: 1.9802 - r_2_score: 0.8589 - val_loss: 8.2785 - val_mean_squared_error: 8.2785 - val_root_mean_squared_error: 2.8772 - val_mean_absolute_error: 2.2426 - val_r_2_score: 0.8003
Epoch 51/100
36/36 [==============================] - 0s 2ms/step - loss: 6.9992 - mean_squared_error: 6.9992 - root_mean_squared_error: 2.6456 - mean_absolute_error: 1.8529 - r_2_score: 0.8452 - val_loss: 9.2189 - val_mean_squared_error: 9.2189 - val_root_mean_squared_error: 3.0363 - val_mean_absolute_error: 2.381

Epoch 73/100
36/36 [==============================] - 0s 2ms/step - loss: 5.4164 - mean_squared_error: 5.4164 - root_mean_squared_error: 2.3273 - mean_absolute_error: 1.6324 - r_2_score: 0.9077 - val_loss: 9.1756 - val_mean_squared_error: 9.1756 - val_root_mean_squared_error: 3.0291 - val_mean_absolute_error: 2.3051 - val_r_2_score: 0.7887
Epoch 74/100
36/36 [==============================] - 0s 2ms/step - loss: 5.5962 - mean_squared_error: 5.5962 - root_mean_squared_error: 2.3656 - mean_absolute_error: 1.7180 - r_2_score: 0.8952 - val_loss: 8.6141 - val_mean_squared_error: 8.6141 - val_root_mean_squared_error: 2.9350 - val_mean_absolute_error: 2.2771 - val_r_2_score: 0.8105
Epoch 75/100
36/36 [==============================] - 0s 2ms/step - loss: 5.4434 - mean_squared_error: 5.4434 - root_mean_squared_error: 2.3331 - mean_absolute_error: 1.6580 - r_2_score: 0.8370 - val_loss: 8.8107 - val_mean_squared_error: 8.8107 - val_root_mean_squared_error: 2.9683 - val_mean_absolute_error: 2.266

Epoch 97/100
36/36 [==============================] - 0s 2ms/step - loss: 4.4435 - mean_squared_error: 4.4435 - root_mean_squared_error: 2.1080 - mean_absolute_error: 1.5004 - r_2_score: 0.9067 - val_loss: 8.8509 - val_mean_squared_error: 8.8509 - val_root_mean_squared_error: 2.9750 - val_mean_absolute_error: 2.2228 - val_r_2_score: 0.7702
Epoch 98/100
36/36 [==============================] - 0s 2ms/step - loss: 4.4606 - mean_squared_error: 4.4606 - root_mean_squared_error: 2.1120 - mean_absolute_error: 1.4999 - r_2_score: 0.9114 - val_loss: 8.8247 - val_mean_squared_error: 8.8247 - val_root_mean_squared_error: 2.9706 - val_mean_absolute_error: 2.2994 - val_r_2_score: 0.7963
Epoch 99/100
36/36 [==============================] - 0s 2ms/step - loss: 4.4265 - mean_squared_error: 4.4265 - root_mean_squared_error: 2.1039 - mean_absolute_error: 1.4640 - r_2_score: 0.8986 - val_loss: 8.4110 - val_mean_squared_error: 8.4110 - val_root_mean_squared_error: 2.9002 - val_mean_absolute_error: 2.223

...

Epoch 93/100
36/36 [==============================] - 0s 2ms/step - loss: 1.7978 - mean_squared_error: 9.2337 - root_mean_squared_error: 3.0387 - mean_absolute_error: 1.7978 - r_2_score: 0.8771 - val_loss: 2.5352 - val_mean_squared_error: 10.6355 - val_root_mean_squared_error: 3.2612 - val_mean_absolute_error: 2.5352 - val_r_2_score: 0.7658
Epoch 94/100
36/36 [==============================] - 0s 2ms/step - loss: 1.7418 - mean_squared_error: 8.8343 - root_mean_squared_error: 2.9723 - mean_absolute_error: 1.7418 - r_2_score: 0.8813 - val_loss: 2.8473 - val_mean_squared_error: 12.8115 - val_root_mean_squared_error: 3.5793 - val_mean_absolute_error: 2.8473 - val_r_2_score: 0.7316
Epoch 95/100
36/36 [==============================] - 0s 2ms/step - loss: 1.8126 - mean_squared_error: 9.5928 - root_mean_squared_error: 3.0972 - mean_absolute_error: 1.8126 - r_2_score: 0.8628 - val_loss: 2.5611 - val_mean_squared_error: 11.5922 - val_root_mean_squared_error: 3.4047 - val_mean_absolute_error: 2.

Epoch 17/100
36/36 [==============================] - 0s 2ms/step - loss: 2.5037 - mean_squared_error: 17.5710 - root_mean_squared_error: 4.1918 - mean_absolute_error: 2.5037 - r_2_score: 0.7476 - val_loss: 2.6439 - val_mean_squared_error: 13.4398 - val_root_mean_squared_error: 3.6660 - val_mean_absolute_error: 2.6439 - val_r_2_score: 0.7754
Epoch 18/100
36/36 [==============================] - 0s 2ms/step - loss: 2.4767 - mean_squared_error: 17.7465 - root_mean_squared_error: 4.2127 - mean_absolute_error: 2.4767 - r_2_score: 0.7353 - val_loss: 2.5740 - val_mean_squared_error: 12.8220 - val_root_mean_squared_error: 3.5808 - val_mean_absolute_error: 2.5740 - val_r_2_score: 0.7933
Epoch 19/100
36/36 [==============================] - 0s 2ms/step - loss: 2.4255 - mean_squared_error: 17.0950 - root_mean_squared_error: 4.1346 - mean_absolute_error: 2.4255 - r_2_score: 0.7519 - val_loss: 2.5639 - val_mean_squared_error: 12.6708 - val_root_mean_squared_error: 3.5596 - val_mean_absolute_error:

Epoch 41/100
36/36 [==============================] - 0s 2ms/step - loss: 2.0722 - mean_squared_error: 14.0268 - root_mean_squared_error: 3.7452 - mean_absolute_error: 2.0722 - r_2_score: 0.7933 - val_loss: 2.3788 - val_mean_squared_error: 10.3870 - val_root_mean_squared_error: 3.2229 - val_mean_absolute_error: 2.3788 - val_r_2_score: 0.8287
Epoch 42/100
36/36 [==============================] - 0s 2ms/step - loss: 2.0607 - mean_squared_error: 13.8039 - root_mean_squared_error: 3.7154 - mean_absolute_error: 2.0607 - r_2_score: 0.7909 - val_loss: 2.3604 - val_mean_squared_error: 10.4504 - val_root_mean_squared_error: 3.2327 - val_mean_absolute_error: 2.3604 - val_r_2_score: 0.8127
Epoch 43/100
36/36 [==============================] - 0s 2ms/step - loss: 2.0338 - mean_squared_error: 13.5563 - root_mean_squared_error: 3.6819 - mean_absolute_error: 2.0338 - r_2_score: 0.8074 - val_loss: 2.5175 - val_mean_squared_error: 11.1427 - val_root_mean_squared_error: 3.3381 - val_mean_absolute_error:

Epoch 65/100
36/36 [==============================] - 0s 2ms/step - loss: 1.8961 - mean_squared_error: 12.5604 - root_mean_squared_error: 3.5441 - mean_absolute_error: 1.8961 - r_2_score: 0.7522 - val_loss: 2.3016 - val_mean_squared_error: 10.0925 - val_root_mean_squared_error: 3.1769 - val_mean_absolute_error: 2.3016 - val_r_2_score: 0.8047
Epoch 66/100
36/36 [==============================] - 0s 2ms/step - loss: 1.8757 - mean_squared_error: 12.1717 - root_mean_squared_error: 3.4888 - mean_absolute_error: 1.8757 - r_2_score: 0.8144 - val_loss: 2.4831 - val_mean_squared_error: 10.9107 - val_root_mean_squared_error: 3.3031 - val_mean_absolute_error: 2.4831 - val_r_2_score: 0.7717
Epoch 67/100
36/36 [==============================] - 0s 2ms/step - loss: 1.9178 - mean_squared_error: 12.4021 - root_mean_squared_error: 3.5217 - mean_absolute_error: 1.9178 - r_2_score: 0.8299 - val_loss: 2.2593 - val_mean_squared_error: 9.5522 - val_root_mean_squared_error: 3.0907 - val_mean_absolute_error: 

Epoch 89/100
36/36 [==============================] - 0s 2ms/step - loss: 1.7467 - mean_squared_error: 10.9873 - root_mean_squared_error: 3.3147 - mean_absolute_error: 1.7467 - r_2_score: 0.8521 - val_loss: 2.2836 - val_mean_squared_error: 10.0248 - val_root_mean_squared_error: 3.1662 - val_mean_absolute_error: 2.2836 - val_r_2_score: 0.7987
Epoch 90/100
36/36 [==============================] - 0s 2ms/step - loss: 1.7771 - mean_squared_error: 11.2160 - root_mean_squared_error: 3.3490 - mean_absolute_error: 1.7771 - r_2_score: 0.7788 - val_loss: 2.2482 - val_mean_squared_error: 9.4148 - val_root_mean_squared_error: 3.0684 - val_mean_absolute_error: 2.2482 - val_r_2_score: 0.8244
Epoch 91/100
36/36 [==============================] - 0s 2ms/step - loss: 1.7481 - mean_squared_error: 10.9643 - root_mean_squared_error: 3.3112 - mean_absolute_error: 1.7481 - r_2_score: 0.8647 - val_loss: 2.2362 - val_mean_squared_error: 9.5288 - val_root_mean_squared_error: 3.0869 - val_mean_absolute_error: 2

In [534]:
#сводная таблица метрик трехуровневых моделей
metric_nn_3

,Layers,Layer_1,Layer_2,Act_func,Loss,Opt,MSE_train,MSE_test,RMSE_train,RMSE_test,MAE_train,MAE_test,R2_score_train,R2_score_test
0,3,128,64,relu,mse,adam,4.035002,8.716541,2.008732,2.952379,1.416167,2.252301,0.922223,0.799543
1,3,128,64,relu,mse,sgd,2.500412,7.617669,1.581269,2.760012,1.098306,2.106887,0.953004,0.820650
2,3,128,64,relu,mse,rmsprop,3.885241,9.112166,1.971102,3.018636,1.462114,2.394183,0.934701,0.819375
3,3,128,64,relu,mae,adam,6.358987,7.848067,2.521703,2.801440,1.366141,2.217632,0.887946,0.837677
4,3,128,64,relu,mae,sgd,7.988771,9.713205,2.826441,3.116601,1.700210,2.446271,0.870827,0.754997
5,3,128,64,relu,mae,rmsprop,8.005769,9.494978,2.829447,3.081392,1.484108,2.295797,0.896788,0.842897
6,3,128,64,linear,mse,adam,3.908277,7.891169,1.976936,2.809122,1.395210,2.117130,0.911059,0.815170
7,3,128,64,linear,mse,sgd,3.199504,8.644803,1.788716,2.940205,1.271967,2.215735,0.941085,0.807616
8,3,128,64,linear,mse,rmsprop,3.863997,7.789046,1.965705,2.790886,1.413392,2.149239,0.927743,0.844802
9,3,128,64,linear,mae,adam,6.197170,7.865866,2.489412,2.804615,1.276605,2.183806,0.898465,0.828197


In [535]:
#лучший показатель MSE на тесте
metric_nn_3.sort_values(by='MSE_test').head(1)

,Layers,Layer_1,Layer_2,Act_func,Loss,Opt,MSE_train,MSE_test,RMSE_train,RMSE_test,MAE_train,MAE_test,R2_score_train,R2_score_test
36,3,64,32,relu,mse,adam,5.859296,7.441838,2.420598,2.727973,1.710657,2.177745,0.819793,0.831548


In [536]:
#лучший показатель RMSE на тесте
metric_nn_3.sort_values(by='RMSE_test').head(1)

,Layers,Layer_1,Layer_2,Act_func,Loss,Opt,MSE_train,MSE_test,RMSE_train,RMSE_test,MAE_train,MAE_test,R2_score_train,R2_score_test
36,3,64,32,relu,mse,adam,5.859296,7.441838,2.420598,2.727973,1.710657,2.177745,0.819793,0.831548


In [537]:
#лучший показатель MAE на тесте
metric_nn_3.sort_values(by='MAE_test').head(1)

,Layers,Layer_1,Layer_2,Act_func,Loss,Opt,MSE_train,MSE_test,RMSE_train,RMSE_test,MAE_train,MAE_test,R2_score_train,R2_score_test
15,3,128,32,relu,mae,adam,9.028551,7.659853,3.004755,2.767644,1.546675,2.048081,0.88929,0.85279


In [538]:
#лучший показатель r2_score на тесте
metric_nn_3.sort_values(by='R2_score_test', ascending=False).head(1)

,Layers,Layer_1,Layer_2,Act_func,Loss,Opt,MSE_train,MSE_test,RMSE_train,RMSE_test,MAE_train,MAE_test,R2_score_train,R2_score_test
23,3,128,32,linear,mae,rmsprop,9.863797,8.507103,3.140668,2.916694,1.600682,2.17769,0.853342,0.853741


**Вывод:** Среди трехуровневых моделей:
1. Лучший показатель MSE имеет модель с 64 нейронами на 1 слое, 32 нейронами на 2 слое, функцией активации relu, функцией ошибок mse и оптимизатором adam.
2. Лучший показатель RMSE имеет модель с 64 нейронами на 1 слое, 32 нейронами на 2 слое, функцией активации relu, функцией ошибок mse и оптимизатором adam.
3. Лучший показатель MAE имеет модель с 128 нейронами на 1 слое, 32 нейронами на 2 слое, функцией активации relu, функцией ошибок mae и оптимизатором adam.
4. Лучшую метрику r2_score имеет модель с 128 нейронами на 1 слое, 32 нейронами на 2 слое, функцией активации linear, функцией ошибок mae и оптимизатором rmsprop.

## 4. Сводная таблица метрик моделей и выводы<a id='section_4'></a>

In [543]:
#результирующая таблица метрик двух- и трехуровневых моделей
metric_nn_result = pd.concat([metric_nn_2, metric_nn_3], axis=0)
#metric_nn_result.sort_values(by='MSE_test')
#metric_nn_result.sort_values(by='RMSE_test')
#metric_nn_result.sort_values(by='MAE_test')
metric_nn_result.sort_values(by='R2_score_test',ascending=False)

,Layers,Layer_1,Layer_2,Act_func,Loss,Opt,MSE_train,MSE_test,RMSE_train,RMSE_test,MAE_train,MAE_test,R2_score_train,R2_score_test
23,3,128,32,linear,mae,rmsprop,9.863797,8.507103,3.140668,2.916694,1.600682,2.177690,0.853342,0.853741
15,3,128,32,relu,mae,adam,9.028551,7.659853,3.004755,2.767644,1.546675,2.048081,0.889290,0.852790
8,3,128,64,linear,mse,rmsprop,3.863997,7.789046,1.965705,2.790886,1.413392,2.149239,0.927743,0.844802
27,3,64,64,relu,mae,adam,8.254122,8.446482,2.872999,2.906283,1.467544,2.186063,0.871728,0.844253
41,3,64,32,relu,mae,rmsprop,10.901978,8.637749,3.301814,2.939005,1.692800,2.167507,0.856408,0.843462
42,3,64,32,linear,mse,adam,5.688736,7.583189,2.385107,2.753759,1.708138,2.129995,0.875138,0.843054
5,3,128,64,relu,mae,rmsprop,8.005769,9.494978,2.829447,3.081392,1.484108,2.295797,0.896788,0.842897
21,3,128,32,linear,mae,adam,7.756202,8.352012,2.784996,2.889985,1.433784,2.098710,0.880329,0.840389
22,3,128,32,linear,mae,sgd,7.898187,7.849099,2.810371,2.801624,1.570856,2.159940,0.891310,0.839581
3,3,128,64,relu,mae,adam,6.358987,7.848067,2.521703,2.801440,1.366141,2.217632,0.887946,0.837677


**Вывод:** лучшие метрики MSE, RMSE, MAE и r2_score имеет трехслойная модель (см. пункт 3).

## Выводы

Проведенный анализ несложных (двух- и трехслойных) нейронных сетей показал, что:

1) лучшие оценки метрики имеет трехслойная модель,

2) лучшие показатели MAE и r2_score достигаются с использование функции потерь mae (c функциями активации relu и linear, оптимизаторами adam и rmsprop соответственно),

3) увеличению качества нейронной сети способствует увеличение числа нейронов в слоях, увеличение числа слоев, 

4) в зависимости от метрики необходимо подбирать оптимальные функцию активации и оптимизатор,

5) лучшие модели по метрикам:\
   MAE -- трехслойная модель (128, 32, 1, функция активации relu, функция ошибок mae и оптимизатор adam),\
   r2_score -- трехслойная модель (128, 32, 1, функция активации linear, функция ошибок mae и оптимизатор rmsprop).